In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import PromptTemplate#
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain import OpenAI
import gradio as gr

In [2]:

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_TOKEN')

embeddings = OpenAIEmbeddings()
db = Chroma(persist_directory="db", embedding_function=embeddings)


Using embedded DuckDB with persistence: data will be stored in: db


In [3]:

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in German:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                 chain_type="stuff",
                                 retriever=db.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)


In [11]:
docs = db.similarity_search(query)
qa_rerank = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True)


In [13]:
query = "Wann hat Prof. Dr. Wildemann einen Verdiensorden erhalten?"
result = qa_rerank({"input_documents": docs, "question": query}, return_only_outputs=True)

In [23]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
If the answer contains of multiple parts, give it in bullet points. 

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In German: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In German:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True, prompt=PROMPT)
query = "Welche Dienstleistungen bietet TCW an?"
result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
result


{'source': 'scraper/html/tcw.de/tcw.de_sitemap.html',
 'intermediate_steps': [{'answer': ' Ich weiß es nicht.', 'score': '0'},
  {'answer': ' TCW bietet Beratungsdienstleistungen, Finanzdienstleistungen und Immobiliendienstleistungen an.',
   'score': '100'},
  {'answer': ' TCW bietet Unternehmensberatung an.', 'score': '100'},
  {'answer': ' TCW bietet Beratungsdienstleistungen in den Bereichen Finanzen, Steuern, Recht und Unternehmensführung an.',
   'score': '100'}],
 'output_text': ' TCW bietet Beratungsdienstleistungen, Finanzdienstleistungen und Immobiliendienstleistungen an.'}

In [6]:
input_query = "Wann hat Prof. Dr. Wildemann einen Verdiensorden erhalten?"

qa({"query":input_query})

{'query': 'Wann hat Prof. Dr. Wildemann einen Verdiensorden erhalten?',
 'result': ' 2006 erhielt Professor Wildemann vom Bayerischen MinisterprÃ¤sidenten den Bayerischen Verdienstorden fÃ¼r seine herausragenden Leistungen in Wissenschaft und Industrie.',
 'source_documents': [Document(page_content='Staatsmedaille des Freistaates Bayern, das Bundesverdienstkreuz 1. Klasse der Bundesrepublik Deutschland und die EhrendoktorwÃ¼rden der UniversitÃ¤ten Klagenfurt, Passau und Cottbus verliehen. Seit 2004 ist er in die Logistik Hall of Fame aufgenommen worden. 2006 erhielt Professor Wildemann vom Bayerischen MinisterprÃ¤sidenten den Bayerischen Verdienstorden fÃ¼r seine herausragenden Leistungen in Wissenschaft und Industrie. Im Jahr 2008 wurde ihm die Ehrennadel der Bundesvereinigung Logistik', metadata={'source': 'scraper/html/tcw.de/tcw.de_unternehmen_sonstiges_prof-wildemann-4.html', 'filename': 'scraper/html/tcw.de/tcw.de_unternehmen_sonstiges_prof-wildemann-4.html', 'page_number': 1, 'c

In [51]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains import ConversationalRetrievalChain


llm = OpenAI(temperature=0)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=db.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [52]:
chat_history = []
query = "Wo hat Prof. Dr. Wildemann studiert?"
result = chain({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

In [54]:
query = "Hat er in seiner Karriere Auszeichnungen erhalten?"
result = chain({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

In [55]:
result

{'question': 'Hat er in seiner Karriere Auszeichnungen erhalten?',
 'chat_history': [('Wo hat Prof. Dr. Wildemann studiert?',
   ' Prof. Dr. Wildemann hat in Aachen und Köln Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) studiert.'),
  ('Hat er in seiner Karriere Auszeichnungen erhalten?',
   ' Yes, Prof. Dr. Wildemann has received awards in his career, including the prestigious German Research Foundation Prize.')],
 'answer': ' Yes, Prof. Dr. Wildemann has received awards in his career, including the prestigious German Research Foundation Prize.'}

In [49]:
from langchain.chains import ConversationalRetrievalChain
qa_conv = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), db.as_retriever(), qa_prompt=PROMPT, chain_type="stuff")
chat_history = []

In [38]:

query = "Wann kam Prof. Dr. Wildemann an die TU München?"
result = qa_conv({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

#query = "Did he mention who she suceeded"
#result = qa_conv({"question": query, "chat_history": chat_history})

In [43]:
query = "Wo hat Prof. Dr. Wildemann studiert?"
result = qa_conv({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

In [46]:
query = "Hat er einen Vedienstorden erhalten?"
result = qa_conv({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))

In [47]:
result

{'question': 'Hat er einen Vedienstorden erhalten?',
 'chat_history': [('Wann kam Prof. Dr. Wildemann an die TU München?', ' 1989'),
  ('Wo hat Prof. Dr. Wildemann studiert?',
   ' Er hat in Aachen und Köln Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) studiert.'),
  ('Hat er einen Vedienstorden erhalten?',
   ' Ja, er hat 2006 vom Bayerischen Ministerpräsidenten den Bayerischen Verdienstorden erhalten.')],
 'answer': ' Ja, er hat 2006 vom Bayerischen Ministerpräsidenten den Bayerischen Verdienstorden erhalten.'}

In [48]:
chat_history

[('Wann kam Prof. Dr. Wildemann an die TU München?', ' 1989'),
 ('Wo hat Prof. Dr. Wildemann studiert?',
  ' Er hat in Aachen und Köln Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) studiert.'),
 ('Hat er einen Vedienstorden erhalten?',
  ' Ja, er hat 2006 vom Bayerischen Ministerpräsidenten den Bayerischen Verdienstorden erhalten.')]

In [ ]:
chat_history = [(query, result["answer"])]
query = "Did he mention who she suceeded"
result = qa({"question": query, "chat_history": chat_history})

In [ ]:

def qa_query(query):
    result = qa({"query": query})
    return result

with gr.Blocks() as tcw_bot:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter question and press enter", show_label=False)
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]
    
    def bot(history):
        response = qa_query(history[-1][0])
        print(response)
        response_result = response["result"]
        metadata = response["source_documents"]
        history[-1][1] = response_result

        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)


tcw_bot.launch(share=False)